In [18]:
!pip install torcheval

In [19]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

# import for NLP
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.utils import to_categorical


from sklearn.metrics import f1_score



In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [21]:
# mount the drive where your dataset is available
from google.colab import drive
drive.mount('/content/drive')
filepath='/content/drive/MyDrive/datasets/multimodal_product_classification/' # add your own path. Where to save the dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
# Load data
X_train = pd.read_csv(filepath+'X_train.csv')
y_train = pd.read_csv(filepath+'Y_train.csv')
X_train=X_train.drop(columns="Unnamed: 0")
y_train=y_train.drop(columns="Unnamed: 0")

In [23]:
# Cleaning and Preprocessing Text
def clean_text(text):
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-ZäöüßÄÖÜ ]', '', text)
    # Convert text to lowercase
    text = text.lower()
    return text

In [24]:
# Apply cleaning function to the 'designation' column
X_train['designation'] = X_train['designation'].fillna('').apply(clean_text)

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['designation'])
sequences = tokenizer.texts_to_sequences(X_train['designation'])

# Padding to max length of text
data = pad_sequences(sequences, maxlen=34)

# Assuming the number of unique words in the tokenizer plus 1 is vocab_size
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)# nearly 70 000 of vocab size, it seems too much

69189


In [25]:
# Split data into training and validation set (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(data, y_train, test_size=0.2,shuffle=False)
y_train=y_train["prdtypecode"].tolist()
y_val=y_val["prdtypecode"].tolist()

In [26]:
# Convert labels to categorical
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Convert to one-hot encoding
y_train_categorical = to_categorical(y_train_encoded)
y_val_categorical = to_categorical(y_val_encoded)


In [27]:
class TextDataset(Dataset):
    def __init__(self, X, Y):
        self.inputs = X
        self.labels = Y

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = torch.tensor(self.inputs[idx]).to(device)
        y = torch.tensor(self.labels[idx], dtype=torch.float).to(device)
        return x, y

In [28]:
embedding_dim = 300
batch_size=64

In [29]:
train_dataset=TextDataset(X_train,y_train_categorical)
train_loader=DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset=TextDataset(X_val,y_val_categorical)
val_loader=DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


# Model definition


In [30]:

# Define the model
class CNN_classifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes):
        super(CNN_classifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_blocks = nn.ModuleList([
            nn.Conv2d(1, 512, (i, embedding_dim), padding=(0, 0))
            for i in range(1,7)
        ])
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(512 * 6, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(1)  # Add channel dimension for Conv2d
        conv_outputs = [nn.functional.relu(conv_block(x)).max(dim=3)[0].max(dim=2)[0] for conv_block in self.conv_blocks]# [0] to get only the values and not the indices ( in pos 1 )
        x = torch.cat(conv_outputs, dim=1)
        # Dense Layer

        # Flatten Layer
        x = x.view(x.size(0), -1)
        # Dropout Layer
        x = self.dropout(x)

        x = self.fc(x)
        return x


In [31]:
# Initialize the model
num_classes = 27
model = CNN_classifier(vocab_size, embedding_dim, num_classes)

# Convert the model to CUDA if available
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
# Print the model summary
print(model)


CNN_classifier(
  (embedding): Embedding(69189, 300)
  (conv_blocks): ModuleList(
    (0): Conv2d(1, 512, kernel_size=(1, 300), stride=(1, 1))
    (1): Conv2d(1, 512, kernel_size=(2, 300), stride=(1, 1))
    (2): Conv2d(1, 512, kernel_size=(3, 300), stride=(1, 1))
    (3): Conv2d(1, 512, kernel_size=(4, 300), stride=(1, 1))
    (4): Conv2d(1, 512, kernel_size=(5, 300), stride=(1, 1))
    (5): Conv2d(1, 512, kernel_size=(6, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=3072, out_features=27, bias=True)
)


In [32]:
train=False

In [33]:
if train:
  # Train the model
  num_epochs = 10  # Adjust as needed
  val_f1=0
  for epoch in tqdm(range(num_epochs)):
      model.train()
      for inputs, labels in tqdm(train_loader,desc=f"Epoch {epoch + 1}/{num_epochs}, Validation F1 Score: {val_f1:.4f}"):
          optimizer.zero_grad()
          outputs = model(inputs)
          outputs=outputs.squeeze(0)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

      model.eval()
      with torch.no_grad():
          val_preds = []
          val_labels = []
          for val_inputs, val_labels_batch in val_loader:
              val_outputs = model(val_inputs)
              val_preds.append(val_outputs.cpu())
              val_labels.append(val_labels_batch.cpu())

      val_preds = torch.cat(val_preds, dim=0)
      val_labels = torch.cat(val_labels, dim=0)

      print(f'{val_labels=}')
      print(f'{torch.argmax(val_preds, dim=1)=}')

      val_f1 = f1_score(torch.argmax(val_labels,dim=1), torch.argmax(val_preds, dim=1), average='macro')


  torch.save(model.state_dict(), '/content/drive/MyDrive/Lessons/Models/multimodal_classification/' + 'epoch{:04}.ckpt'.format(epoch))


# Test part

In [34]:
X_test = pd.read_csv(filepath+'X_test.csv')
X_test=X_test.drop(columns="Unnamed: 0")
X_test['designation'] = X_test['designation'].fillna('').apply(clean_text)
sequences = tokenizer.texts_to_sequences(X_test['designation'])

# Padding to max length of text
data = pad_sequences(sequences, maxlen=34)


In [35]:
print(data)

[[   0    0    0 ... 3534   47 5881]
 [   0    0    0 ...   45   29 1062]
 [   0    0    0 ...  178 3672 5143]
 ...
 [   0    0    0 ... 1434   73   11]
 [   0    0    0 ... 1771 3144 3034]
 [   0    0    0 ... 1205    3  126]]


In [36]:
model = CNN_classifier(vocab_size, embedding_dim, num_classes)
model.load_state_dict(torch.load('/content/drive/MyDrive/Lessons/Models/multimodal_classification/checkpoint.ckpt'))
device=("cpu")
model.to(device)
model.eval()

CNN_classifier(
  (embedding): Embedding(69189, 300)
  (conv_blocks): ModuleList(
    (0): Conv2d(1, 512, kernel_size=(1, 300), stride=(1, 1))
    (1): Conv2d(1, 512, kernel_size=(2, 300), stride=(1, 1))
    (2): Conv2d(1, 512, kernel_size=(3, 300), stride=(1, 1))
    (3): Conv2d(1, 512, kernel_size=(4, 300), stride=(1, 1))
    (4): Conv2d(1, 512, kernel_size=(5, 300), stride=(1, 1))
    (5): Conv2d(1, 512, kernel_size=(6, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=3072, out_features=27, bias=True)
)

In [ ]:
val_preds = []
with torch.no_grad():
  for input in tqdm(data):
      input=torch.from_numpy(input)
      input=input.to(device)
      input=input.unsqueeze(0)
      val_outputs = model(input)
      val_preds.append(val_outputs.cpu())

  0%|          | 0/13812 [00:00<?, ?it/s]

[]
[tensor([[ -6.8443,  -4.1299, -15.2575,  -9.4349,  -5.9231, -15.1380,  -7.3063,
           2.7895,  -2.5416,  -1.3768, -20.0264,  -8.9849,   3.4223,  -6.1708,
         -12.7622, -10.0252, -12.4229, -14.7674,  -6.2869,   6.7471,  -3.4938,
          -6.6636,  -3.0675,  -6.1381,  -6.3998,   7.8777, -18.4465]])]
[tensor([[ -6.8443,  -4.1299, -15.2575,  -9.4349,  -5.9231, -15.1380,  -7.3063,
           2.7895,  -2.5416,  -1.3768, -20.0264,  -8.9849,   3.4223,  -6.1708,
         -12.7622, -10.0252, -12.4229, -14.7674,  -6.2869,   6.7471,  -3.4938,
          -6.6636,  -3.0675,  -6.1381,  -6.3998,   7.8777, -18.4465]]), tensor([[  8.0141,  -1.1993, -12.8132,  -0.8425,   4.3100,  23.4400,   0.2249,
           0.6917,   8.7410, -12.5405, -12.0476, -19.5450, -12.5174, -10.5859,
           2.1588, -16.6609,  -8.2309, -13.7878, -16.9886, -13.5795,  -8.4528,
          -6.3888,  -9.7156, -21.2492, -10.1857,  -7.6661, -19.2970]])]
[tensor([[ -6.8443,  -4.1299, -15.2575,  -9.4349,  -5.9231, -15.1380

In [ ]:
val_preds=np.argmax(val_preds,dim=1)

In [ ]:
np.savetxt("foo.csv", a, delimiter=",")